In [1]:
#モジュールインポート
import sys
sys.path.append('..')
sys.path
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#テキストデータの前処理

text = 'You say goodbye and I say hello.'

text = text.lower()
text = text.replace('.',' .')
text

'you say goodbye and i say hello .'

In [3]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [4]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [5]:
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [6]:
#関数化
def preprocess(text):
    text = text.lower()
    text = text.replace('.',' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus,word_to_id,id_to_word

### 2.3.4 共起行列

In [7]:
text = 'You say goodbye and i say hello.'
corpus,word_to_id,id_to_word = preprocess(text)

print(corpus)
print(id_to_word) #語彙数が7つとわかった。

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [8]:
#手動での共起行列の作成
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0],
], dtype=np.int32)

In [9]:
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]])

In [10]:
#自動での共起行列の作成
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size +1 ):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] +=1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id,right_word_id] +=1
    
    return co_matrix

In [11]:
C = create_co_matrix(corpus,7,1)

### 2.3.5ベクトル間の類似度

In [12]:
#コサイン類似度の実装
def cos_similarity(x,y, eps=1e-8):
    nx = x/ (np.sqrt(np.sum(x**2))+eps)
    ny = y/ (np.sqrt(np.sum(y**2))+ eps)
    return np.dot(nx,ny)

In [13]:
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0,c1))

0.7071067691154799


### 類似単語のランキング表示

In [14]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        priint('%s is not found' % query)
        return

    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    count = 0
    for i in (-1* similarity).argsort():
        if id_to_word[i] == query:
            continue
        print('%s:%s' % (id_to_word[i],similarity[i]))
        
        count += 1
        if count >= top:
            return

In [15]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query]you
goodbye:0.7071067691154799
i:0.7071067691154799
hello:0.7071067691154799
say:0.0
and:0.0
